[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Regressor - Polyfit

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/10/2022 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/IntroductionMachineLearningSystemEngineers/RegressorPolyFit.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import make_moons
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from scipy.spatial.distance import cdist

# Misc
import datetime
import os
from platform import python_version
import random
import warnings
import yaml

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

In [ ]:
# Configuration
%matplotlib inline

warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


In [ ]:
# Parameters

# Data Generation
numSamples  = 50
noiseStd    = 0.3

vP = np.array([0.25, 2, 5])
polynomDeg = 2

# Data Visuzalization
gridNoiseStd = 0.05
numGridPts = 250

In [ ]:
# Auxiliary Functions

def PlotPolyFit( vX: np.ndarray, vY: np.ndarray, vP: np.ndarray = None, P: int = 1, numGridPts: int = 1001, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None ):

    if hA is None:
        hF, hA = plt.subplots(1, 2, figsize = figSize)
    else:
        hF = hA[0].get_figure()

    numSamples = len(vY)

    # Polyfit
    vW    = np.polyfit(vX, vY, P)
    
    # MSE
    vHatY = np.polyval(vW, vX)
    MSE   = np.linalg.norm(vY - vHatY) ** 2 / numSamples
    
    # Plot
    xx  = np.linspace(np.floor(np.min(vX)), np.ceil(np.max(vX)), numGridPts)
    yy  = np.polyval(vW, xx)

    hA[0].plot(vX, vY, '.r', ms = 10, label = '$y_i$')
    hA[0].plot(xx, yy, 'b',  lw = 2,  label = '$\hat{f}(x)$')
    hA[0].set_title (f'$P = {P}$\nMSE = {MSE}')
    hA[0].set_xlabel('$x$')
    # hA[0].axis(lAxis)
    hA[0].grid()
    hA[0].legend()
    
    hA[1].stem(vW[::-1], label = 'Estimated')
    if vP is not None:
        hA[1].stem(vP[::-1], linefmt = None, markerfmt = 'D', label = 'Ground Truth')
    hA[1].set_title('Coefficients')
    hA[1].set_xlabel('$w$')
    hA[1].legend()

    # return hF



## Generate Data

$$ y_{i} = f \left( x_{i} \right) + \epsilon_{i} $$

Where

$$ f \left( x \right) = \frac{1}{2} x^{2} + 2x + 5 $$

In [ ]:
def f( vX ):
    # return 0.25 * (vX ** 2) + 2 * vX + 5
    return np.polyval(vP, vX)

vX = np.linspace(-2, 2, numSamples, endpoint = True) + (gridNoiseStd * np.random.randn(numSamples))
vN = noiseStd * np.random.randn(numSamples)
vY = f(vX) + vN


### Plot Data

In [ ]:
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA.plot(vX, vY, '.r', ms = MARKER_SIZE_DEF, label = r'$y_i = \frac{2}{3}x_i^2 + 2x_i + 5 + \epsilon_i$')
hA.set_xlabel('$x$')
hA.legend()
hA.grid()

## Train Polyfit Regressor

$$\arg\min_{\boldsymbol{w}}\left\Vert \boldsymbol{y}-\boldsymbol{\Phi}\boldsymbol{w}\right\Vert _{2}^{2}$$

Where

$$\boldsymbol{\Phi} = \left[\begin{matrix}1 & x_{1} & x_{1}^{2} & \cdots & x_{1}^{P}\\
1 & x_{2} & x_{2}^{2} & \cdots & x_{2}^{P}\\
\vdots & \vdots & \vdots &  & \vdots\\
1 & x_{N} & x_{N}^{2} & \cdots & x_{N}^{P}
\end{matrix}\right]$$

In [ ]:
# Polynomial Fit

vW = np.polyfit(vX, vY, polynomDeg)
vW[::-1]

### Plot Regressor for Various Polynomial Degrees

In [ ]:
hPolyFit = lambda P: PlotPolyFit(vX, vY, P = P)
pSlider = IntSlider(min = 0, max = 31, step = 1, value = 0, layout = Layout(width = '30%'))
interact(hPolyFit, P = pSlider)
plt.show()

### Sensitivity to Support

We'll show the effect of the support, given a number of sample on the estimated weights (_Coefficients_).

In [ ]:
vN = 20 * noiseStd * np.random.randn(numSamples)

def GenDataByRadius( valR: float = 1.0 ):

    P = 2

    vX = np.linspace(-valR, valR, numSamples, endpoint = True)
    vY = f(vX) + vN
    
    PlotPolyFit(vX, vY, vP = vP, P = P)
    


In [ ]:
rSlider = FloatSlider(min = 0.1, max = 50.0, step = 0.1, value = 0.1, layout = Layout(width = '30%'))
interact(GenDataByRadius, valR = rSlider)
plt.show()